In [90]:
from abc import ABCMeta, abstractmethod
import numpy as np
import matplotlib.pyplot as plt
from typing import *
from typing import Tuple
import itertools as it


def compute_distance(point1: np.ndarray, point2: np.ndarray) -> float:
    sq = 0
    for x, y in zip(point1, point2):
        sq += (x - y)**2
    return np.sqrt(sq)

def angle(point1: np.ndarray, point2: np.ndarray, point3: np.ndarray) -> float:
        ba = point2 - point1
        bc = point2 - point3
        out =  np.degrees(np.arccos(np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))))
        return out

class Cluster(metaclass=ABCMeta):
    def __init__(self, center_coordinates: np.ndarray, shape: np.ndarray, step: float):
        self.center = center_coordinates
        if any([size == 0 for size in shape]):
            raise ValueError('Shape of a cluster can not contain 0')
        self.shape = shape
        self.step = step
        self.volume = None

    def __str__(self):
        return '{}-dimensional cluster\n ' \
               'with center in {}\n ' \
               'and with shape {}'.format(self.center.shape[0], self.center, self.shape*2)
    def __mul__(self, num: int):
        self.shape = self.shape*num
        return self.shape
    def __add__(self, coordinate: np.ndarray):
        self.center = np.add(self.center, coordinate)
        return self.center
    def __len__(self):
        return len(self.center)
    def __lt__(self, other):
        return self.volume < other.volume
    def __gt__(self, other):
        return self.volume > other.volume
    def __le__(self, other):
        return self.volume <= other.volume
    def __ge__(self, other):
        return self.volume >= other.volume
    def __eq__(self, other):
        return self.volume == other.volume
    def __contains__(self, item: np.ndarray):
        return self.includes(item)

    @property
    def center(self):
        return self
    @center.setter
    def center(self, center_coordinates: np.ndarray):
        self._center = center_coordinates
    @center.getter
    def center(self):
        return self._center

    @property
    def shape(self):
        return self
    @shape.setter
    def shape(self, shape: np.ndarray):
        self._shape = shape
    @shape.getter
    def shape(self):
        return self._shape

    @property
    def step(self):
        return self
    @step.setter
    def step(self, value):
      self._step = value
    @step.getter
    def step(self):
        return self._step

    @property
    def volume(self):
        return self
    @volume.setter
    def volume(self, value):
      self._volume = value
    @volume.getter
    def volume(self):
        return self._volume

    @abstractmethod
    def includes(self, point: np.ndarray) -> bool:
        pass
    @abstractmethod
    def compute_volume(self):
        pass


class HyperEllipseCluster(Cluster):
    def __init__(self, center_coordinates: np.ndarray, shape: np.ndarray, step: float = 0.001):
        super().__init__(center_coordinates, shape, step)
        self.volume = self.volume_with_int()

        self.__features = self.focuses(self)
    def __str__(self):
        return '{}-dimensional elliptic cluster\n ' \
               'with center in {}\n ' \
               'and with shape {}'.format(self.center.shape[0], self.center, self.shape*2)
    def __mul__(self, num: int):
        self.shape = self.shape*num
        self.__features = self.focuses(self)
        return self.shape
    def __add__(self, coordinate: np.ndarray):
        self.center = np.add(self.center, coordinate)
        self.__features = self.focuses(self)
        return self.center
    def __iter__(self):
        self.n = 0
        self.c = self.center
        return self
    def __next__(self):
        # recursive solution impossible (max recursion depth is reached)
        while self.n < (1/self.step)**self.center.shape[0]:
            self.n += 1
            self.c = np.array([
                c + (s * self.step) * (
                        self.n % (1/self.step)**i // (1/self.step)**(i-1)
                ) for c, s, i in zip(
                    self.center, self.shape, range(self.center.shape[0], 0, -1)
                )
            ])
            if self.c in self:
                return self.c
        else:
            raise StopIteration

    @staticmethod
    def focuses2(self, combination) -> Tuple[np.ndarray, np.ndarray, float, Tuple[int, int]]:
        x, y = combination
        if self.shape[x] > self.shape[y]:
            a = self.shape[x]
            b = self.shape[y]
            a_index = x
            b_index = y
        else:
            a = self.shape[y]
            b = self.shape[x]
            a_index = y
            b_index = x
        c = np.sqrt(np.square(a) - np.square(b))
        focus1, focus2 = list(), list()
        b_coordinates = list()
        for i in range(len(self)):
            if i == b_index:
                b_coordinates.append(self.center[i] + b)
            else:
                b_coordinates.append(self.center[i])
            if i == a_index:
                focus1.append(self.center[i] - c)
                focus2.append(self.center[i] + c)
            else:
                focus1.append(self.center[i])
                focus2.append(self.center[i])
        focus1, focus2, b_coordinates = np.array(focus1), np.array(focus2), np.array(b_coordinates)
        if all(focus1 == focus2):
            r1 = compute_distance(focus1, b_coordinates)
            r2 = 0
        else:
            r1 = compute_distance(focus1, b_coordinates)
            r2 = compute_distance(focus2, b_coordinates)
        return focus1, focus2, r1+r2, (a_index, b_index)

    @staticmethod
    def focuses(self) -> Dict[str, Union[List[tuple], np.ndarray]]:
        mix_indexes = list(it.combinations(range(self.shape.shape[0]), 2))
        focuses1, focuses2, focal_radius_sums, coordinate_indexes = list(), list(), list(), list()
        for i, j in mix_indexes:
            focus1, focus2, focal_radius_sum, coordinate_index = self.focuses2(
                self,
                (i, j)
            )
            focuses1.append(focus1)
            focuses2.append(focus2)
            focal_radius_sums.append(focal_radius_sum)
            coordinate_indexes.append(coordinate_index)
        return {
            'focuses1': focuses1,
            'focuses2': focuses2,
            'focal_radius_sums': focal_radius_sums,
            'combination': mix_indexes,
            'coordinate_indexes': coordinate_indexes
        }

    def includes(self, point: np.ndarray) -> bool:
        for comb, focus1, focus2, focal_radius_sum, coordinate_indexes in\
                zip(
                    self.__features['combination'],
                    self.__features['focuses1'],
                    self.__features['focuses2'],
                    self.__features['focal_radius_sums'],
                    self.__features['coordinate_indexes']
                ):
            x, y = coordinate_indexes
            point_projection1 = list(self.center.copy())
            point_projection1[x] = point[x]
            point_projection2 = list(self.center.copy())
            point_projection2[y] = point[y]
            r1 = compute_distance(focus1, np.array(point_projection1))
            r2 = compute_distance(focus2, np.array(point_projection2))
            print(r1+r2, focal_radius_sum)
            if focal_radius_sum < r1+r2:
                return False

        return True

    def compute_volume(self):
        dv = np.prod(self.shape*self.step)
        v = 0
        for i in self:
            v += dv
        return v*(2**self.center.shape[0])

    def volume_with_int(self):
        parallelepiped = np.prod(self.shape)
        dimensional_transition = 2*np.pi
        if self.shape.shape[0]%2 != 0:
            spherical_ratio = 2
            for i in range(1, self.shape.shape[0], 2):
                spherical_ratio *= dimensional_transition/(i + 2)
        else:
            spherical_ratio = 1
            for i in range(0, self.shape.shape[0], 2):
                spherical_ratio *= dimensional_transition/(i + 2)
        return parallelepiped*spherical_ratio

class ClustersStatistic(object):
    def __init__(self, clusters: List[HyperEllipseCluster]):
        for i, j in list(it.combinations(range(len(clusters)), 2)):
            if clusters[i].center.shape != clusters[j].center.shape:
                raise ValueError('Clusters must have equal dimensions')
            if all(clusters[i].center == clusters[j].center):
                raise ValueError('Clusters {} and {} have equal center coordinates. All the next computations have no any sense'.format(i, j))
        self.clusters = clusters
        self.distances = {
            'distance_between_{}_and_{}'.format(i, j):
            compute_distance(clusters[i].center, clusters[j].center)
            for i, j in list(it.combinations(range(len(clusters)), 2))
        }
        self.edges = {
            'edges_for_{}_and_{}_common_axis'.format(i, j):
            self.compute_edges(clusters[i], clusters[j])
            for i, j in list(it.combinations(range(len(clusters)), 2))

        }
    @staticmethod
    def compute_edges(cluster1: Cluster, cluster2: Cluster, iters: int = 100):
        def find_edge(cluster, step_price, r, iters):
            diff = r
            for i in range(iters):
                r_check = r
                diff = diff/2
                now = cluster.center + step_price*r
                if now in cluster:
                    while now in cluster:
                        r += diff
                        now = cluster.center + step_price*r
                        if r_check == r:
                            return now
                else:
                    while now not in cluster:
                        r -= diff
                        now = cluster.center + step_price*r
                        if r_check == r:
                            return now
            return cluster.center + step_price*r

        step_price1 = cluster2.center - cluster1.center
        step_price2 = cluster1.center - cluster2.center
        r1 = np.sqrt(np.sum(cluster1.shape**2))
        r2 = np.sqrt(np.sum(cluster2.shape**2))
        return (
            find_edge(cluster1, step_price1, r1, iters),
            find_edge(cluster2, step_price2, r2, iters)
        )




cls1 = HyperEllipseCluster(np.array([0, 0, 0]), np.array([5, 5, 5]))
# cls2 = HyperEllipseCluster(np.array([4, 4]), np.array([1, 1]))
print(np.array([0, 0, 5]) in cls1)
# cls3 = HyperEllipseCluster(np.array([0, -1, 0, 1, 0, -1, 0, 1]), np.array([1, 1, 0, 1, 0, -1, 0, 1]))
# cls_stat = ClustersStatistic([cls1, cls2])
# print(cls_stat.edges)
# print(cls1.volume)



0.0 5.0
5.0000001 5.0
False
